# Transfer learning

In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize, Compose, Resize

batch_size = 64
# Define the necessary transformations
preprocess = Compose([
    ToTensor(),
    Resize((300, 300)),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageFolder(root="data/horse-or-human", transform=preprocess)
test_dataset = ImageFolder(root="data/validation-horse-or-human", transform=preprocess)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [16]:
# dispositivo de entrenamiento
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Usando: {device}")

Usando: cuda


In [17]:
# bucle de entrenamiento

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # modo entrenamiento
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # mover si es necesario
        X = X.to(device)
        y = y.to(device)
        # forward
        # prediccion y costo
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        
        loss, current = loss.item(), batch * batch_size + len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# funcion de pruebas
def test_loop(dataloader, model, loss_fn):
    # modo evaluacion
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    # evaluacion del modelo con torch.no_grad
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Usando un modelo Pre-entrenado
En este caso usaremos el modelo VGG16 pre entrenado en el dataset de IMAGENET

In [18]:
vgg = models.alexnet()
print(vgg)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [19]:
from torchvision import models

class CustomVgg(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.backbone = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        self.backbone.classifier[6] = nn.Linear(in_features=4096, out_features=2)

    def forward(self, x):
        output = self.backbone(x)
        return output
    
model = CustomVgg().to(device)
print(model)

KeyboardInterrupt: 

In [ ]:
learning_rate = 1e-4
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# bucle principal
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.783590  [   64/ 1027]
loss: 0.414034  [  128/ 1027]
loss: 0.184262  [  192/ 1027]
loss: 0.030569  [  256/ 1027]
loss: 0.003162  [  320/ 1027]
loss: 0.018954  [  384/ 1027]
loss: 0.000053  [  448/ 1027]
loss: 0.000003  [  512/ 1027]
loss: 0.062409  [  576/ 1027]
loss: 0.000000  [  640/ 1027]
loss: 0.163338  [  704/ 1027]
loss: 0.079153  [  768/ 1027]
loss: 0.000000  [  832/ 1027]
loss: 0.000004  [  896/ 1027]
loss: 0.000410  [  960/ 1027]
loss: 0.199902  [ 1024/ 1027]
loss: 0.000000  [ 1027/ 1027]


/home/pepe/miniconda3/envs/cv_unifranz/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000338 

Epoch 2
-------------------------------
loss: 0.000000  [   64/ 1027]
loss: 0.000000  [  128/ 1027]
loss: 0.000000  [  192/ 1027]
loss: 0.000000  [  256/ 1027]
loss: 0.040102  [  320/ 1027]
loss: 0.000000  [  384/ 1027]
loss: 0.000072  [  448/ 1027]
loss: 0.002080  [  512/ 1027]
loss: 0.000054  [  576/ 1027]
loss: 0.001256  [  640/ 1027]
loss: 0.000886  [  704/ 1027]
loss: 0.000000  [  768/ 1027]
loss: 0.000000  [  832/ 1027]
loss: 0.002976  [  896/ 1027]
loss: 0.000376  [  960/ 1027]
loss: 0.000000  [ 1024/ 1027]
loss: 0.000002  [ 1027/ 1027]
Test Error: 
 Accuracy: 95.7%, Avg loss: 0.216875 

Epoch 3
-------------------------------
loss: 0.000000  [   64/ 1027]
loss: 0.001537  [  128/ 1027]
loss: 0.000000  [  192/ 1027]
loss: 0.000000  [  256/ 1027]
loss: 0.000000  [  320/ 1027]
loss: 0.000000  [  384/ 1027]
loss: 0.000000  [  448/ 1027]
loss: 0.000000  [  512/ 1027]
loss: 0.000000  [  576/ 1027]
loss: 0.000000  [  640/ 1027]
loss:

# Fine tuning

In [22]:
class CustomVgg2(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.backbone = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        # frozen layers
        for param in self.backbone.parameters():
                param.requires_grad = False
        self.backbone.classifier[6] = nn.Linear(in_features=4096, out_features=2)

    def forward(self, x):
        output = self.backbone(x)
        return output
    
model = CustomVgg2().to(device)

In [23]:
learning_rate = 1e-4
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# bucle principal
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.599366  [   64/ 1027]
loss: 0.563507  [  128/ 1027]
loss: 0.574543  [  192/ 1027]
loss: 0.520833  [  256/ 1027]
loss: 0.493749  [  320/ 1027]
loss: 0.440332  [  384/ 1027]
loss: 0.454804  [  448/ 1027]
loss: 0.480743  [  512/ 1027]
loss: 0.440270  [  576/ 1027]
loss: 0.390819  [  640/ 1027]
loss: 0.366702  [  704/ 1027]
loss: 0.388749  [  768/ 1027]
loss: 0.362336  [  832/ 1027]
loss: 0.348019  [  896/ 1027]
loss: 0.335840  [  960/ 1027]
loss: 0.361615  [ 1024/ 1027]
loss: 0.432611  [ 1027/ 1027]
Test Error: 
 Accuracy: 99.6%, Avg loss: 0.359390 

Epoch 2
-------------------------------
loss: 0.326969  [   64/ 1027]
loss: 0.290676  [  128/ 1027]
loss: 0.276044  [  192/ 1027]
loss: 0.301342  [  256/ 1027]
loss: 0.263382  [  320/ 1027]
loss: 0.224802  [  384/ 1027]
loss: 0.259421  [  448/ 1027]
loss: 0.251849  [  512/ 1027]
loss: 0.245954  [  576/ 1027]
loss: 0.262392  [  640/ 1027]
loss: 0.224817  [  704/ 1027]
loss: 0.198153  [  768/ 1027